# Pleque - impact angle and related functionality demonstration

Branch feature/filedline_spreading

In [ ]:
%pylab notebook 

In [ ]:
# Use this function to acess COMPASS-U database
from pleque.io.compass import cudb
# Use this function to load testing equilibrium 
from pleque.tests.utils import load_testing_equilibrium

from pleque.core.coordinates import Coordinates
import matplotlib.pyplot as plt
import numpy as np


In [ ]:

eq = load_testing_equilibrium(4)

In [ ]:
eq.plot_overview()
plt.show()

In [ ]:
# You can see, there is wrong geometry in the database at the moment: 
eq.plot_geometry()

In [ ]:
# Coordinates instance of first wall: 
fw = eq.first_wall

# Midplane separatrix-lcfs distance: 
r_mid = eq.r_mid(psi_n = 1, grid=False)
mapped_clearance = fw.r_mid - r_mid


In [ ]:
opts = {'vmin': 0, 'vmax': 0.03, 'edgecolors': 'none'}

#ax = axs[0]
plt.figure(figsize=(4.5,6))
ax = plt.gca()

# plot LCFS: 
eq.lcfs.plot(color='k', ls='--')
cl = ax.scatter(fw.R, fw.Z, c=mapped_clearance, **opts)
im = plt.colorbar(mappable=cl, ax=ax, label='Mapped clearance [m]')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')



## Magnetic field on the FW: 

In [ ]:
B_wall = eq.B_abs(fw)


In [ ]:
plt.figure(figsize=(4.5,6))
ax = plt.gca()

# plot LCFS: 
eq.lcfs.plot(color='k', ls='--')
# opts = {'vmin': 6.0, 'vmax': 10.0, 'edgecolors': 'none'}
cl = ax.scatter(fw.R, fw.Z, c=B_wall)
im = plt.colorbar(mappable=cl, ax=ax, label='Magnetic feild on FW [T]')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

## Refine the wall using the new methon of resampling: 



In [ ]:
#number of points to resample the limiter
npoints=1000

# Resample using new method
coords2=eq.first_wall.resample2(npoints)

# use dir to print all atributes od the Coordinates class
print(dir(coords2))


 ## Calculate cosinus of impact angle

In [ ]:
??coords2.impact_angle_cos()

In [ ]:
import pleque.utils.flux_expansions as flux_expansions
flux_expansions.impact_angle_cos??

In [ ]:
# get normal vector
normal_vecs=coords2.normal_vector()

# get normalised bvec
bvec=eq.Bvec_norm(coords2)

vec=np.linspace(0,0.2,2)

print(bvec.shape)
print(normal_vecs.shape)
# get impact angle sinus

impcos=coords2.impact_angle_sin()

#get only poloidal projection
# get impact angle sinus
impcospol=coords2.impact_angle_sin_pol_projection()


In [ ]:
## Display limiter normal angles vs impact angles

In [ ]:
fign, axn = plt.subplots()

axn.plot(eq.first_wall.R, eq.first_wall.Z, lw=2, color='c')

axn.set_aspect('equal')

#plot normal vectors
for i in range(0,npoints-1):
    axn.plot(coords2.R[i]+normal_vecs[0, i]*vec,
             coords2.Z[i]+normal_vecs[1, i]*vec, color='k')
    
#plot b vectors
    
for i in range(0,npoints-1):
    axn.plot(coords2.R[i]+bvec[0, i]*vec, 
             coords2.Z[i]+bvec[1, i]*vec,color='r')
    

grid = eq.grid(resolution=[1e-3, 2e-3], dim="step")

cl = axn.contour(grid.R, grid.Z, grid.psi_n, 400, alpha=0.5)

axn.set_xlim(0.43,0.55)
axn.set_ylim(-0.37,-0.3)

## Display limiter impact angle cosine

In [ ]:
fig, ax=plt.subplots()

ax.set_aspect('equal')
#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))

cl = ax.scatter(coords2.R, coords2.Z, c=np.abs(impcos))
im = plt.colorbar(mappable=cl, ax=ax, label='Impact angle sinus')
ax.contour(grid.R, grid.Z, grid.psi_n, 50, alpha=0.5)

ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

plt.show()

In [ ]:
fig,ax=plt.subplots()

ax.set_aspect('equal')
#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))

cl = ax.scatter(coords2.R, coords2.Z, c=np.abs(impcospol))
im = plt.colorbar(mappable=cl, ax=ax, label='Impact angle pol projection sine')

ax.contour(grid.R, grid.Z, grid.psi_n, 60, alpha=0.5)
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

plt.show()

In [ ]:
# Poloidal field exp. coef
fx =eq.poloidal_mag_flux_exp_coef(coords2)
# Poloidal field exp. coef with projection to the target
fx_eff=eq.effective_poloidal_mag_flux_exp_coef(coords2)

In [ ]:
in_angle = coords2.impact_angle_sin_pol_projection()

plt.figure()
plt.plot(coords2.theta, fx, '--', label='fx')
plt.plot(coords2.theta, in_angle, label='imp angle sin')
plt.plot(coords2.theta, fx/in_angle, label='fx/imp.angl.sin')
plt.plot(coords2.theta, fx_eff, '--', label='fx_eff')
plt.legend()

In [ ]:
fig,ax=plt.subplots()


eq.plot_overview(ax=ax)
ax.set_aspect('equal')

#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))
opts = {'vmin': 0, 'vmax': 10, 'edgecolors': 'none'}

cl = ax.scatter(coords2.R, coords2.Z, c=ratio, **opts)
im = plt.colorbar(mappable=cl, ax=ax, label='Magnetic_field_ratio_poloidal')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

ax.set_xlim(0.37,0.6)
ax.set_ylim(-0.37,-0.3)

plt.show()


fig,ax=plt.subplots()

ax.set_aspect('equal')

#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))
opts = {'vmin': 1, 'vmax': 10, 'edgecolors': 'none'}

cl = ax.scatter(coords2.R, coords2.Z, c=ratio, **opts)

eq.separatrix.plot(color='C2', alpha=0.4)
im = plt.colorbar(mappable=cl, ax=ax, label='Effective magnetic field expansion coef.')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

ax.set_xlim(0.37,0.6)
ax.set_ylim(-0.37,-0.3)

plt.show()



plt.figure()
plt.plot(coords2.theta, ratio, label='Magnetic field expansion coef.')
plt.plot(coords2.theta, ratio2, label='Effective magnetic field expansion coef.')
plt.axvline()
plt.legend()
plt.ylim(0, 20)

In [ ]:
flux_exp_coef=ratio/np.abs(impcos)

In [ ]:
fig,ax=plt.subplots()

ax.set_aspect('equal')
#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))
opts = {'vmin': 0, 'vmax': 0.0025, 'edgecolors': 'none'} #

cl = ax.scatter(coords2.R, coords2.Z, c=flux_exp_coef,marker='s',**opts)
im = plt.colorbar(mappable=cl, ax=ax, label='Total coeff cos* flux expansion')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

ax.set_xlim(0.37,0.6)
ax.set_ylim(-0.37,-0.3)

ax.contour(grid.R, grid.Z, grid.psi_n, 600, alpha=0.5)

plt.show()

In [ ]:
flux_exp_coef=ratio2*np.abs(impcospol)

In [ ]:
fig,ax=plt.subplots()

ax.set_aspect('equal')
#cmap=plt.get_cmap('jet')

#norm_cos=(impcos-np.amin(impcos))/(np.amax(impcos)-np.amin(impcos))
opts = {'vmin': 0, 'vmax': 0.25, 'edgecolors': 'none'} #

cl = ax.scatter(coords2.R[:-1], coords2.Z[:-1], c=flux_exp_coef,marker='s',**opts)
im = plt.colorbar(mappable=cl, ax=ax, label='Total coeff pol variant')
ax.set_aspect('equal')
ax.set_ylabel('Z [m]')
ax.set_xlabel('R [m]')

ax.set_xlim(0.37,0.6)
ax.set_ylim(-0.37,-0.3)

ax.contour(grid.R, grid.Z, grid.psi_n, 600, alpha=0.5)

plt.show()